In [ ]:
pip install torch transformers pandas openpyxl xlsxwriter

In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer

# Load the CodeT5+ model and tokenizer
checkpoint = "Salesforce/codet5p-110m-embedding"
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to(device)

# Load the Excel file from the specified path
df = pd.read_excel("/content/AST_C.xlsx")
print("Data loaded successfully:")
print(df.head())  # Display the first few rows of the DataFrame

# Function to encode data in batches
def encode_data_batch(df, model, tokenizer, device, start_row, end_row):
    embeddings = []
    for i in range(start_row, end_row):
        code = df['C_AST'][i]  # Use the 'java' column
        # Convert code to string and handle potential NaN values
        code = str(code) if not pd.isnull(code) else ''
        inputs = tokenizer.encode(code, return_tensors="pt", max_length=512, truncation=True).to(device)
        with torch.no_grad():
            embedding = model(inputs)[0].cpu().numpy()  # Get fixed-size embeddings
        embeddings.append(embedding)
    return embeddings

# Process the data in batches
batch_size = 300
all_embeddings = []

start_row = 0
while start_row < len(df):
    end_row = min(start_row + batch_size, len(df))
    embeddings_batch = encode_data_batch(df, model, tokenizer, device, start_row, end_row)
    all_embeddings.extend(embeddings_batch)
    start_row = end_row

# Convert to numpy array and reshape if necessary
embeddings_array = np.vstack(all_embeddings)

# Create DataFrame from embeddings
embedding_columns = [f'c_AST_only_embedding_{i}' for i in range(embeddings_array.shape[1])]
embeddings_df = pd.DataFrame(embeddings_array, columns=embedding_columns)

# Save embeddings to Excel with a neatly formatted sheet
output_path = "/content/c_AST_embeddings.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    embeddings_df.to_excel(writer, index=False, sheet_name="Embeddings")
    workbook = writer.book
    worksheet = writer.sheets["Embeddings"]

    # Set column widths for neat formatting
    for i, column in enumerate(embeddings_df.columns):
        max_len = max(embeddings_df[column].astype(str).map(len).max(), len(column))
        worksheet.set_column(i, i, max_len + 2)  # Adjust the padding as needed

print(f"Embeddings have been saved to {output_path}")


Data loaded successfully:
                                               C_AST
0  (translation_unit (preproc_include path: (syst...
1  (translation_unit (preproc_include path: (syst...
2  (translation_unit (preproc_include path: (syst...
3  (translation_unit (preproc_include path: (syst...
4  (translation_unit (preproc_include path: (syst...
Embeddings have been saved to /content/c_AST_embeddings.xlsx


In [ ]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 2.7 MB/s eta 0:00:00


In [ ]:
# Step 1: Install required libraries
!pip install hdbscan umap-learn openpyxl

import pandas as pd
import umap

# Step 2: Load the embeddings
embeddings_df = pd.read_excel("/content/c_AST_embeddings.xlsx")

# Step 3: Select the embedding data
# Adjust this if you want to select specific columns
embedding_data = embeddings_df.iloc[:, :].values  # Use all embedding columns

# Step 4: Apply UMAP for dimensionality reduction to 2 components
umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=15, random_state=42)
umap_embeddings = umap_model.fit_transform(embedding_data)

# Step 5: Create a DataFrame from UMAP embeddings
umap_df = pd.DataFrame(umap_embeddings, columns=['UMAP_1', 'UMAP_2', 'UMAP_3', 'UMAP_4', 'UMAP_5', 'UMAP_6', 'UMAP_7', 'UMAP_8', 'UMAP_9', 'UMAP_10','UMAP_11', 'UMAP_12', 'UMAP_13', 'UMAP_14', 'UMAP_15'])

# Step 6: Save the UMAP results to a new Excel file
output_path = "/content/C_ast_umap15_embeddings.xlsx"
umap_df.to_excel(output_path, index=False)

print(f"UMAP embeddings have been saved to {output_path}")

/usr/local/lib/python3.10/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP embeddings have been saved to /content/C_ast_umap15_embeddings.xlsx


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap2_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 8


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap3_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 8


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap4_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 9


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap5_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 9


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap6_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 8


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap7_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 8


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap8_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 10


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap9_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 8


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap10_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 9


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap15_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
# You can adjust eps based on your data characteristics
dbscan = DBSCAN(eps=0.5, min_samples=5)  # min_samples can be set to a low value
cluster_labels = dbscan.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 8


In [ ]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap2_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 3


In [ ]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap3_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 3


In [ ]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap4_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 3


In [ ]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap5_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 3


In [ ]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap6_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 3


In [ ]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap7_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 3


In [ ]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap8_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 3


In [ ]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap9_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 4


In [ ]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap10_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 3


In [ ]:
import pandas as pd
import hdbscan
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap15_embeddings.xlsx"
df = pd.read_excel(file_path)

# Assuming the embeddings are in columns and are numeric
# Convert DataFrame to numpy array for HDBSCAN
embeddings = df.values

# Apply HDBSCAN clustering
clusterer = hdbscan.HDBSCAN(min_cluster_size=5)  # You can adjust min_cluster_size as needed
cluster_labels = clusterer.fit_predict(embeddings)

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)

print(f"Number of clusters formed: {num_clusters}")

Number of clusters formed: 3


In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN
import numpy as np

# Load the Excel file containing the embeddings
file_path = "/content/C_ast_umap8_embeddings.xlsx"
df = pd.read_excel(file_path)

# Convert DataFrame to numpy array for DBSCAN
embeddings = df.values

# Apply DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Adjust 'eps' and 'min_samples' as needed
cluster_labels = dbscan.fit_predict(embeddings)

# Add cluster labels to the DataFrame
df['Cluster_Label'] = cluster_labels

# Save the DataFrame with cluster labels to a new Excel file
output_file_path = "/content/result.xlsx"
df.to_excel(output_file_path, index=False)

print(f"Clustered data saved to: {output_file_path}")

# Find the number of clusters formed (excluding noise)
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
print(f"Number of clusters formed: {num_clusters}")

Clustered data saved to: /content/result.xlsx
Number of clusters formed: 10


In [ ]:
pip install torch transformers pandas openpyxl xlsxwriter

In [47]:
import pandas as pd

# Load the dataset
file_path = '/content/result.xlsx'
df = pd.read_excel(file_path)

# Check if 'Cluster_Label' exists in the DataFrame
if 'Cluster_Label' in df.columns:
    # Count distinct elements and their occurrences
    distinct_counts = df['Cluster_Label'].value_counts()
    print(distinct_counts)
else:
    print("Column 'Cluster_Label' does not exist in the dataset.")

Cluster_Label
 7    29
 0    26
 1    23
 5    13
 3    12
 4    12
 2    10
-1     7
 6     7
 9     6
 8     5
Name: count, dtype: int64
